In [1]:
# DO ANY PASSES OVERLAP

In [4]:
# Import necessary libraries
import ephem
import math
from datetime import timedelta
from datetime import datetime


# get_passes() function definition
def get_passes(observer, tle, start_time, num_passes=None, duration=None):
    """Config obs and sat, Return pass data for all passes in given interval.

    Arguments:
    observer -- 4 element list containing desired [name,lat,lon,alt]
    tle -- 3 element list containing desired tle [line0,line1,line2]
    start_time -- ephem.date string formatted 'yyyy/mm/dd hr:min:sec'
    num_passes -- integer number of desired passes (defualt None)
    duration -- float number of hours or fraction of hours (default None)

    Specify either num_passes or duration.
    If both, use num_passes.
    If neither, find passes for next 24 hours.
    """

    obs_name, obs_lat, obs_lon, obs_alt = observer
    tle_line0, tle_line1, tle_line2 = tle

    # Set up location of observer
    ground_station = ephem.Observer()
    ground_station.name = obs_name                # name string
    ground_station.lon = obs_lon                  # in degrees (+E)
    ground_station.lat = obs_lat                  # in degrees (+N)
    ground_station.elevation = obs_alt            # in meters
    ground_station.date = ephem.date(start_time)  # in UTC

    # Read in most recent satellite TLE data
    sat = ephem.readtle(tle_line0, tle_line1, tle_line2)

    contacts = []

    if num_passes is not None and duration is None:
        # if only num_passes specified
        try:
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(days = set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
                contacts.append(pass_data)
        except ValueError:
            # No (more) visible passes
            pass
        return contacts

    if num_passes is None and duration is not None:
        # if only duration specified
        try:
            end_time = ephem.date(ground_station.date+duration*ephem.hour)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
        except ValueError:
            # No (more) visible passes
            pass
        return pass_data

    if num_passes is not None and duration is not None:
        # if both are specified, use num_passes
        try:
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
        except ValueError:
            # No (more) visible passes
            pass
        return pass_data

    if num_passes is None and duration is None:
        # if neither are specified, get passes for the next 24 hours
        try:
            end_time = ephem.date(ground_station.date+1)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
        except ValueError:
            # No (more) visible passes
            pass
        return pass_data


In [5]:
from itertools import islice
data = []
with open('amateur.txt') as f:
    while True:
        #an iterator that returns the next N lines and stops
        tripleline = islice(f, 3)
        #loop over these N lines, removing trailing spaces and \n
        tle = [x.rstrip() for x in tripleline]

        #only accept complete data
        #the end of the file *should* have len(tle)==0 but
        #this also handles extra junk at the end
        if len(tle) == 3:
            data.append(tle)
        else:
            break


In [9]:
vu = ['Valparaiso University', '41.4639', '-87.0439', 245.089]
start_time = '2017/6/8 12:00:00'
num_passes = 1
duration = None
vu_passes = {}
i = 0

for tle in data:
    # use NORAD ID as key for each satellite
    # value is list of passes, where each pass is a dictionary of data
    vu_passes[data[i][2][2:7]] = get_passes(vu, tle, start_time, num_passes=num_passes, duration=duration)
    i = i + 1

In [46]:
for sat, passes in vu_passes.items():
    for obs in passes:
        print('NORAD ID:', sat, 'START:', obs['start'], 'END:', obs['end'])


NORAD ID: 22826 START: Thu Jun  8 12:02:07 2017 END: Thu Jun  8 12:08:08 2017
NORAD ID: 32789 START: Thu Jun  8 14:27:17 2017 END: Thu Jun  8 14:38:00 2017
NORAD ID: 27844 START: Thu Jun  8 12:18:07 2017 END: Thu Jun  8 12:33:52 2017
NORAD ID: 39436 START: Thu Jun  8 18:00:57 2017 END: Thu Jun  8 18:11:25 2017
NORAD ID: 40905 START: Thu Jun  8 13:26:58 2017 END: Thu Jun  8 12:03:53 2017
NORAD ID: 39153 START: Thu Jun  8 15:58:37 2017 END: Thu Jun  8 16:06:02 2017
NORAD ID: 39446 START: Thu Jun  8 12:08:11 2017 END: Thu Jun  8 12:15:33 2017
NORAD ID: 26931 START: Thu Jun  8 19:02:24 2017 END: Thu Jun  8 19:06:29 2017
NORAD ID: 35935 START: Thu Jun  8 19:32:07 2017 END: Thu Jun  8 19:43:37 2017
NORAD ID: 35932 START: Thu Jun  8 18:39:56 2017 END: Thu Jun  8 18:42:26 2017
NORAD ID: 33493 START: Thu Jun  8 12:58:06 2017 END: Thu Jun  8 13:05:10 2017
NORAD ID: 40908 START: Thu Jun  8 12:18:44 2017 END: Thu Jun  8 12:30:45 2017
NORAD ID: 39433 START: Thu Jun  8 14:55:07 2017 END: Thu Jun  8 